In [1]:
cd /Users/benrosand/oct-qiskit-pulse


/Users/benrosand/oct-qiskit-pulse


In [2]:
from qiskit import IBMQ, pulse, assemble
import numpy as np
from qiskit.providers.aer.pulse.system_models.hamiltonian_model import HamiltonianModel
from qiskit.pulse import Schedule, Play
from qiskit.tools import job_monitor
from qutip import sigmax, tensor, identity, Qobj
from qutip.control.pulseoptim import optimize_pulse_unitary
from qutip.qip.device import Processor
from src.qutip_helper import convert_qutip_ham
import qutip.logging_utils as logging
import qutip
import math
import qutip.control.pulseoptim as cpo

from qutip.control import *
from qutip import *

import datetime

In [3]:
from src.schedule_helper import channel_finder
# subsystem_list=[0,1]
subsystem_list=[0]
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibmq_armonk')

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
backend.configuration().hamiltonian['vars']['wq0']
ham = convert_qutip_ham(backend, subsystem_list, False)
ham


/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'H_c': {'D0': Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
  Qobj data =
  [[0.         0.03593737 0.        ]
   [0.03593737 0.         0.05082312]
   [0.         0.05082312 0.        ]]},
 'iH_c': {},
 'H_d': Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
 Qobj data =
 [[ 0.          0.          0.        ]
  [ 0.         31.25541787  0.        ]
  [ 0.          0.         62.51083574]]}

In [5]:
# a= Qobj([[0,1,0],
#             [0,0,math.sqrt(2)],
#             [0,0,0]])

# adag = a.dag()

# sigmax_3 = a + adag
# sigmay_3 = complex(0,1) * (adag - a)
# # sigmaz_3 = 



/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
H_d = drift_hamiltonian = ham['H_d']
H_c = control_hamiltonians = [ham['H_c']['D0'] * 1e9] 
# control_hamiltonians = [sigmax_3]

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
sigmax_goal = Qobj([[0,1,0],
                    [1,0,0],
                    [0,0,1]])

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
# Duration of each timeslot
dt = 0.02
# List of evolution times to try
evo_times = [1, 3, 6]
n_evo_times = len(evo_times)
evo_time = evo_times[0]
n_ts = int(float(evo_time) / dt)
#Empty list that will hold the results for each evolution time
results = list()

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
# n_ts = 1280

sigmax_3 = Qobj([[0,1,0],[1,0,math.sqrt(2)], [0, math.sqrt(2),0]])

# dt = backend.configuration().dt
# n_ts = 640 
# n_ts = 320
# evo_time = np.pi
# evo_time = n_ts * dt * 1e9
# drift_hamiltonian = drift_hamiltonian * 1e9
# control_hamiltonians = [control * 1e9 for control in control_hamiltonians

fid_err_targ=1e-20
max_iter=2000
max_wall_time=120
min_grad=1e-20
p_type='LIN'
alg='GRAPE'

log_level = logging.WARN
U_0 = identity(3)
U_targ = sigmax_goal

result = optimize_pulse_unitary(drift_hamiltonian, control_hamiltonians, U_0, U_targ, n_ts,
                                evo_time,
                                fid_err_targ=fid_err_targ, min_grad=min_grad,
                                max_iter=max_iter, max_wall_time=max_wall_time,
                                out_file_ext=None, init_pulse_type=p_type,
                                log_level=log_level, gen_stats=True, alg=alg,
                                amp_lbound=-1, amp_ubound=1, phase_option='PSU')
result.evo_full_final

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[-0.29590144-0.68457814j -0.53002558+0.18540569j  0.15334614+0.32400259j]
 [-0.52999618+0.18530067j -0.07967596-0.22779329j -0.7472157 +0.26114815j]
 [ 0.15338007+0.32409469j -0.74713574+0.26124272j -0.18836111-0.45770919j]]

In [36]:
# Duration of each timeslot
dt = 0.05
# List of evolution times to try
evo_times = [1, 3, 6]
n_evo_times = len(evo_times)
evo_time = evo_times[0]
n_ts = int(float(evo_time) / dt)
#Empty list that will hold the results for each evolution time
results = list()

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
optim = cpo.create_pulse_optimizer(H_d, H_c, U_0, U_targ, n_ts, evo_time, 
                amp_lbound=-5.0, amp_ubound=5.0, 
                fid_err_targ=fid_err_targ, min_grad=min_grad, 
                max_iter=max_iter, max_wall_time=max_wall_time, 
                optim_method='fmin_l_bfgs_b',
                method_params={'max_metric_corr':20, 'accuracy_factor':1e8},
                dyn_type='UNIT', 
                fid_params={'phase_option':'PSU'},
                init_pulse_type=p_type, 
                log_level=log_level, gen_stats=True)

# **** get handles to the other objects ****
optim.test_out_files = 0
dyn = optim.dynamics
dyn.test_out_files = 0
p_gen = optim.pulse_generator
                


/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
n_ctrls=1
f_ext=None

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# init_amps[:, j]
p_gen.gen_pulse()
n_ts

dyn


/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'init_amps' is not defined

In [40]:
for i in range(n_evo_times):
    print(i)
    # Generate the tau (duration) and time (cumulative) arrays
    # so that it can be used to create the pulse generator
    # with matching timeslots
    dyn.init_timeslots()
    if i > 0:
        # Create a new pulse generator for the new dynamics
        p_gen = pulsegen.create_pulse_gen(p_type, dyn)
        
    #Generate different initial pulses for each of the controls
    init_amps = np.zeros([n_ts, n_ctrls])
    if (p_gen.periodic):
        phase_diff = np.pi / n_ctrls
        for j in range(n_ctrls):
            init_amps[:, j] = p_gen.gen_pulse(start_phase=phase_diff*j)
    elif (isinstance(p_gen, pulsegen.PulseGenLinear)):
        for j in range(n_ctrls):
            p_gen.scaling = float(j) - float(n_ctrls - 1)/2
            init_amps[:, j] = p_gen.gen_pulse()
    elif (isinstance(p_gen, pulsegen.PulseGenZero)):
        for j in range(n_ctrls):
            p_gen.offset = sf = float(j) - float(n_ctrls - 1)/2
            init_amps[:, j] = p_gen.gen_pulse()
    else:
        # Should be random pulse
        for j in range(n_ctrls):
            init_amps[:, j] = p_gen.gen_pulse()
    
    dyn.initialize_controls(init_amps)
    
    # Save initial amplitudes to a text file
    if f_ext is not None:
        pulsefile = "ctrl_amps_initial_" + f_ext
        dyn.save_amps(pulsefile)
        print("Initial amplitudes output to file: " + pulsefile)

    print("***********************************")
    print("\n+++++++++++++++++++++++++++++++++++")
    print("Starting pulse optimisation for T={}".format(evo_time))
    print("+++++++++++++++++++++++++++++++++++\n")
    result = optim.run_optimization()
    results.append(result)

    # Save final amplitudes to a text file
    if f_ext is not None:
        pulsefile = "ctrl_amps_final_" + f_ext
        dyn.save_amps(pulsefile)
        print("Final amplitudes output to file: " + pulsefile)
    
    # Report the results
    result.stats.report()
    print("Final evolution\n{}\n".format(result.evo_full_final))
    print("********* Summary *****************")
    print("Final fidelity error {}".format(result.fid_err))
    print("Final gradient normal {}".format(result.grad_norm_final))
    print("Terminated due to {}".format(result.termination_reason))
    print("Number of iterations {}".format(result.num_iter))
    print("Completed in {} HH:MM:SS.US".format(
            datetime.timedelta(seconds=result.wall_time)))
    
    if i+1 < len(evo_times):
        # reconfigure the dynamics for the next evo time
        evo_time = evo_times[i+1]
        n_ts = int(float(evo_time) / dt)
        dyn.tau = None
        dyn.evo_time = evo_time

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
0
***********************************

+++++++++++++++++++++++++++++++++++
Starting pulse optimisation for T=1
+++++++++++++++++++++++++++++++++++


------------------------------------
---- Control optimisation stats ----
**** Timings (HH:MM:SS.US) ****
Total wall time elapsed during optimisation: 0:00:00.135305
Wall time computing Hamiltonians: 0:00:00.005210 (3.85%)
Wall time computing propagators: 0:00:00.098497 (72.80%)
Wall time computing forward propagation: 0:00:00.001062 (0.78%)
Wall time computing onward propagation: 0:00:00.001048 (0.77%)
Wall time computing gradient: 0:00:00.007368 (5.45%)

**** Iterat

ValueError: could not broadcast input array from shape (20) into shape (60)

In [41]:
result.evo_full_final

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[-1.03595846e-06+4.50504539e-06j  6.31975005e-01-7.74988769e-01j
   1.29569578e-05-4.12541871e-07j]
 [ 6.31975005e-01-7.74988769e-01j  4.20430971e-06-1.92134076e-06j
   2.20278126e-06+1.27750371e-05j]
 [ 1.29569578e-05-4.12542000e-07j  2.20278139e-06+1.27750371e-05j
   6.31993547e-01-7.74973648e-01j]]

In [42]:


c_ops = []
U_f_numerical = propagator(result.H_t, times[-1], c_ops, args={})
U_f_numerical

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'OptimResult' object has no attribute 'H_t'

In [43]:
from qutip import propagator

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
